## Interval Analysis

In [ ]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,)), transforms.Lambda(torch.flatten)]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,)), transforms.Lambda(torch.flatten)]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
class Network(nn.Sequential):
    def __init__(self):
        in_size = 28*28
        classes = 10

        super().__init__(
            nn.Linear(in_size, 200),
            nn.ReLU(),
            nn.Linear(200, classes)
        )

#model = nn.Sequential(Normalize(), Net())
model = Network()

model = model.to(device)
model.train()


Network(
  (0): Linear(in_features=784, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=10, bias=True)
)

In [52]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [53]:
train_model(model, 15)
test_model(model)

Epoch 1/15, Loss: 0.593
Epoch 2/15, Loss: 0.297
Epoch 3/15, Loss: 0.248
Epoch 4/15, Loss: 0.215
Epoch 5/15, Loss: 0.190
Epoch 6/15, Loss: 0.170
Epoch 7/15, Loss: 0.155
Epoch 8/15, Loss: 0.142
Epoch 9/15, Loss: 0.130
Epoch 10/15, Loss: 0.121
Epoch 11/15, Loss: 0.113
Epoch 12/15, Loss: 0.106
Epoch 13/15, Loss: 0.099
Epoch 14/15, Loss: 0.093
Epoch 15/15, Loss: 0.088
Accuracy on images: 97.07


### Write the interval analysis for the simple model

In [59]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation
from bound_propagation import BoundModelFactory, HyperRectangle

factory = BoundModelFactory()
bound_model = factory.build(model)

epsilons = np.linspace(0.01, 0.1, 10)
for eps in epsilons:
  correct = 0
  total = 0
  for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    mask = predicted == labels

    correct_images = images[mask]
    correct_labels = labels[mask]
    input_bounds = HyperRectangle.from_eps(correct_images, eps)
    ibp_bounds = bound_model.ibp(input_bounds)
    for i in range(len(correct_labels)):
      target = correct_labels[i].item()
      verified = True
      for j in range(10):
        if j != target:
          if ibp_bounds.lower[i, target] <= ibp_bounds.upper[i, j]:
            verified = False
            break
      if verified: correct += 1
  
  print(f"epsilon: {eps:.2f}, verified accuracy: {correct/total * 100}")


epsilon: 0.01, verified accuracy: 79.49000000000001
epsilon: 0.02, verified accuracy: 28.37
epsilon: 0.03, verified accuracy: 5.5
epsilon: 0.04, verified accuracy: 1.0999999999999999
epsilon: 0.05, verified accuracy: 0.18
epsilon: 0.06, verified accuracy: 0.01
epsilon: 0.07, verified accuracy: 0.0
epsilon: 0.08, verified accuracy: 0.0
epsilon: 0.09, verified accuracy: 0.0
epsilon: 0.10, verified accuracy: 0.0
